# Converting CVAT to be added to Coco2017 dataset


### First we need to convert the CVAT Yolo 1.1 style (class number) so it can be add it on top of coco dataset 

In [95]:
import os
import shutil

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
# Find all the new classes (folders in coco-additions)
additional_classes = os.listdir(coco_additions_path)  

# New objects
for obj in additional_classes:
  new_obj_path=os.path.normpath(os.path.join(coco_additions_path,obj))
  try: 
    # Try to copy the original train.txt into a train_original.txt
    train_txt_file=os.path.normpath(os.path.join(new_obj_path,"train.txt"))
    shutil.copyfile(train_txt_file, os.path.normpath(os.path.join(new_obj_path,"train_original.txt")))
  except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'c:\\Users\\justm\\Documents\\OID_YoloV5\\yolov5\\datasets\\coco-additions\\finished\\train.txt'


### Now we change the class number of all the coco-additions (prompt user for input)

In [96]:
import os 
coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
additional_classes = os.listdir(coco_additions_path)


for obj in additional_classes:
  new_obj_path=os.path.join(coco_additions_path,obj)

  try:
    # Ask for a new class number for certain object
    obj_train_data = os.path.join(new_obj_path,"obj_train_data") # get the specific file 
    additional_obj_train_data = [os.path.normpath(os.path.join(obj_train_data,file))  for file in os.listdir(obj_train_data) if  file.endswith(".txt")] # find txt in new object 
    class_number  = int(input(f"[{obj}]Please enter a the new class number as an integer"))

    # For all text files (all the labels) replace the label number (row[0]) with new class_number 
    for text_file in additional_obj_train_data: 
      annotations = [] 
      with open(text_file) as f: 
          for line in f:
            labels = line.split() 
            if (not labels[0].isdigit()): #  Issue where the label is not a digit (not a class number)
              raise Exception(text_file, "Label is not a digit (possibly still a label not a number)") 

            labels[0] = class_number # Change the class number
            newline = str(labels[0]) + " " + str(labels[1]) + " " + str(labels[2]) + " " + str(labels[3]) + " " + str(labels[4])
            line = line.replace(line, newline)
            annotations.append(line)
          f.close()

      # Write all the labels into that text file
      with open(text_file, "w") as outfile:
        for line in annotations:
            outfile.write(line)
            outfile.write("\n")
        outfile.close()

  except Exception as e: # case the folder doesn't have data (moved / edited)
    print(new_obj_path, "Does not have 'obj_train_data' folder in it, skipped")


c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/electric_hoverboard Does not have 'obj_train_data' folder in it, skipped
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/electric_scooter Does not have 'obj_train_data' folder in it, skipped
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/electric_unicycle Does not have 'obj_train_data' folder in it, skipped
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/finished Does not have 'obj_train_data' folder in it, skipped


### Changing format of text file that holds location of images 

Change the  `coco_additions/<object>/train.txt` file so that it matches the coco2017 dataset ```datasets/coco2017/train2017.txt``` style

CVAT yolo style  = ```data/obj_train_data/hoverboard_0.jpg```

CoCo train2017.txt style = ```./images/train2017/000000109622.jpg```

In [97]:
import os
import shutil

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
additional_classes = os.listdir(coco_additions_path)

# For all classes 
for obj in additional_classes:
  new_obj_path=os.path.normpath(os.path.join(coco_additions_path,obj))

  try:
    # Copy the original to train.txt (so we don't get duplicates) 
    train_txt_file=os.path.normpath(os.path.join(new_obj_path,"train.txt"))
    shutil.copyfile(os.path.normpath(os.path.join(new_obj_path,"train_original.txt")),train_txt_file)

    # We change the CVAT stype to YOLO 
    annotations = []
    with open(train_txt_file) as f: # open text file
        for line in f: # for each line
          image_name = line.split("/")[-1].strip() # removes \n at the end as well
          image_line = f"./images/train2017/{image_name}"
          annotations.append(image_line) # List of string for each line (replace first value with class number entered)
        f.close()
    with open(train_txt_file, "w") as outfile:
      for line in annotations:
          outfile.write(line)
          outfile.write("\n")
      outfile.close()
  except Exception as e: 
    print(new_obj_path, "Does not have 'train.txt' folder in it, skipped")


c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco-additions\finished Does not have 'train.txt' folder in it, skipped


### Merge the CVAT train.txt into the coco2017.txt

BUGS: allows duplicates of the same, however yoloV5 training model should be able to deal with it

In [ ]:
import os
import shutil
# copy train2017_original.txt to train2017.txt
coco2017_train_path = os.path.join(os.getcwd(),"datasets/coco2017/train2017.txt") # opens the train.txt path
shutil.copyfile(os.path.normpath(os.path.join(os.getcwd(),"datasets/coco2017/train2017_original.txt")),coco2017_train_path) # copy original to train 


coco2017_file_obj = open(coco2017_train_path, 'a') # open folder to append stuff

coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
additional_classes = os.listdir(coco_additions_path)
for obj in additional_classes: # for all classes 
  new_obj_path=os.path.normpath(os.path.join(coco_additions_path,obj))

  try: 
    # Each class we open the train.txt file
    train_txt_file=os.path.normpath(os.path.join(new_obj_path,"train.txt"))
    with open(train_txt_file) as f: # open text file
          for line in f: # for each line
            coco2017_file_obj.write(line) # Append the image path to the .txt file
          f.close() # close file when done
    print(new_obj_path, "Has been appended to datasets/coco2017/train2017.txt")
  except:
    print(new_obj_path, "Does not have 'train.txt' folder in it, skipped")
coco2017_file_obj.close()

: 

### Move images to coco dataset

Once everything is sure to correct 

Checklist 
1. The classes labels are all changed (no duplicate class number 80+)
2. The path format of the CVAT images are changed to match the coco dataset (```./images/train2017/000000109622.jpg```)
3. datasets/coco2017/train2017.txt has the new images appended to the bottom 
   
Now we need to move the images and the labels into their corresponding folder in coco2017 dataset 

*WARNING*

Once moved it will be almost impossible to move the files out as they will be mixed within the CoCo2017 dataset

In [99]:
import os 
coco_additions_path = os.path.join(os.getcwd(),"datasets/coco-additions/")
additional_classes = os.listdir(coco_additions_path)

# all classes
for obj in additional_classes:
  new_obj_path=os.path.join(coco_additions_path,obj)
  try:
    obj_train_data = os.path.join(new_obj_path,"obj_train_data") # get the specific file 
    jpg_names = [os.path.splitext(file)[0]  for file in os.listdir(obj_train_data) if  file.endswith(".jpg")] # holds all the file names (no extensions)
    coco_image_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/coco2017/images/train2017"))
    coco_label_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/coco2017/labels/train2017"))
    # C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
    # C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\labels\train2017
    
    for image in jpg_names:
      # Now we have the file name we add the extensions
      label_file_name = image + ".jpg"
      jpg_file_name = image + ".txt"
      
      # move from "\datasets\coco-addition\object\obj_train_data" to "\datasets\coco2017\labels\train2017"
      shutil.move(os.path.normpath(os.path.join(obj_train_data,label_file_name)),os.path.normpath(os.path.join(coco_image_path,label_file_name)))
      shutil.move(os.path.normpath(os.path.join(obj_train_data,jpg_file_name)),os.path.normpath(os.path.join(coco_label_path,jpg_file_name)))

    print(f"[Moved] {obj} images and labels to coco paths {coco_image_path}")
  except Exception as e: 
    print(new_obj_path, "Does not have 'obj_train_data' folder in it, skipped")




[Moved] bicycle_helmet images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
[Moved] electric_hoverboard images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
[Moved] electric_scooter images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
[Moved] electric_unicycle images and labels to coco paths c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/finished Does not have 'obj_train_data' folder in it, skipped


# Selecting a custom amount of training samples

Limit CoCo2017 dataset to a certain amount 

In [101]:
limit = 10000 # Only keep this many photos

coco_train_txt = os.path.join(os.getcwd(),"datasets/coco2017/train2017.txt")


keep_lines = []
with open(coco_train_txt) as f: # open text file
    for line in f: # for each line
      file_name = os.path.splitext(os.path.basename(line))[0]
      if file_name.isdigit(): # is a digit number means its from coco 
        if len(keep_lines) < limit:
          keep_lines.append(file_name)
      else:
        keep_lines.append(file_name)


with open(os.path.join(os.getcwd(),"datasets/coco2017/" + f"train{limit}.txt"), 'w') as outfile:
  for file_name in keep_lines:
      line_value = f"./images/train2017/{file_name}.jpg"
      outfile.write(line_value)
      outfile.write("\n")
outfile.close()


print(new_obj_path, f"Has been appended to train{limit}.txt")

c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco2017/train2017.txt
c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets/coco-additions/finished Has been appended to train1000000.txt


# Converting OIDV4 downloaded to CVAT format

OIDv4 is great because you can download already labelled images from the OpenImage dataset

1. Check that the labels are in *class numbers* rather than *class names*
2. Use "yolov5_convert_annotations.py" to convert to class number

### This function below moves the converted OIDv4 Data into a format for COCO to be used in
object_name = name of the folder that will be moved
it will be converted to lower case with spaces replaced to underscores

In [3]:
import os 
import shutil
import time

object_name = "truck"
new_object_name = object_name.lower().replace(" ","_")

oidv_obj = os.path.join( os.path.dirname(os.getcwd()),"OIDv4_ToolKit","OID","Dataset","train",object_name)
if os.path.exists(oidv_obj):
  coco_addition_folder=os.path.join(os.getcwd(),"datasets","coco-additions",new_object_name)
  if not os.path.exists(coco_addition_folder):
    os.makedirs(coco_addition_folder)
    os.makedirs(os.path.join(coco_addition_folder,"obj_train_data"))
    obj_train_data_path = os.path.join(coco_addition_folder,"obj_train_data")
  # else: 
  #   raise Exception(f"Object already exists in {coco_addition_folder}")
    
  train_files = os.listdir(oidv_obj)
  for file_names in [file for file in train_files if file.endswith(".jpg")]: # for files that are jpg
    # print(os.path.join(oidv_obj,file_names))
    source = os.path.normpath(os.path.join(oidv_obj,file_names))
    dest = os.path.normpath(os.path.join(obj_train_data_path,file_names))
    shutil.move(source,dest)

  label_files_path = os.path.join(oidv_obj,"Label_yolo")
  label_files = os.listdir(label_files_path)
  for file_names in [file for file in label_files if file.endswith(".txt")]: # for files that are txt
    source = os.path.normpath(os.path.join(label_files_path,file_names))
    dest = os.path.normpath(os.path.join(obj_train_data_path,file_names))
    shutil.move(source,dest)

with open(os.path.join(coco_addition_folder,'obj.names'), 'w') as f:
    f.write(new_object_name)




c:\Users\justm\Documents\OID_YoloV5\OIDv4_ToolKit\OID\Dataset\train\truck\Label_yolo


### Afterwards you need need to generated a new train.txt and train.txt original 

These txt files are needed if you want to combine them with the coco dataset

In [24]:
import time
dataset_name=input("Please enter your dataset name")
train_text_format = f"./data/{object_name}/images"

# what is the object name (as the image / label will be renamed to this)
object_name = "motorcycle"
obj_train_path = os.path.join(os.getcwd(),"datasets",dataset_name, object_name , "obj_train_data")

###  Rename file
train_files = os.listdir(obj_train_path)
annotations= []
for file_names in [file for file in train_files if file.endswith(".jpg")]: # for each line
  image_line = f"data/obj_train_data/{file_names}"
  annotations.append(image_line) # List of string for each line (replace first value with class number entered)

# for all images rename to <class_name>_number
for idx,files_names in enumerate(annotations):
  # obj_train_path # folder path
  txt_name = os.path.splitext(os.path.basename(files_names))[0]  + ".txt"
  jpg_name = os.path.basename(files_names)
  try: 
    os.rename(obj_train_path + f"/{txt_name}", obj_train_path + f"/{object_name}_{idx+1}" + ".txt" )
    os.rename(obj_train_path + f"/{jpg_name}", obj_train_path + f"/{object_name}_{idx+1}" + ".jpg") 
  except Exception as e:
    pass
    # print(f'File {txt_name} missing')
time.sleep(3)

### Read the files inside again after rename then write to train.txt
train_files = os.listdir(obj_train_path)
annotations= []
for file_names in [file for file in train_files if file.endswith(".jpg")]: # for each line
  image_line = f"{train_text_format}/{file_names}"
  annotations.append(image_line) # List of string for each line (replace first value with class number entered)

train_original_Path = os.path.join(os.getcwd(),"datasets",dataset_name, object_name , "train_original.txt")
print(train_original_Path)
with open(train_original_Path, 'w') as outfile:
  for line in annotations:
      outfile.write(line)
      outfile.write("\n")
outfile.close()
shutil.copy(os.path.join(os.getcwd(),train_original_Path), os.path.join(os.getcwd(),"datasets",dataset_name, object_name , "train.txt"))




c:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\airport\motorcycle\train_original.txt


'c:\\Users\\justm\\Documents\\OID_YoloV5\\yolov5\\datasets\\airport\\motorcycle\\train.txt'

In [16]:
### Separate training 
import os
import shutil

source_folder = obj_train_path
destination_folder_images = os.path.abspath(os.path.join(obj_train_path, os.pardir,"images"))
destination_folder_txt = os.path.abspath(os.path.join(obj_train_path, os.pardir,"labels"))

if not os.path.exists(destination_folder_images):
  os.mkdir(destination_folder_images)
if not os.path.exists(destination_folder_txt):
  os.mkdir(destination_folder_txt)

# fetch all files
for file_name in os.listdir(source_folder):
    source = os.path.join(source_folder, file_name)
    if  os.path.isfile(source):
      if file_name.endswith(".jpg"):
          destination = os.path.join(destination_folder_images,file_name)
          shutil.move(source, destination)
      elif file_name.endswith(".txt"):
          destination = os.path.join(destination_folder_txt,file_name)
          shutil.move(source, destination)


# Other functions that may be of use

### Delete all images that dont start with 0000 
Thus not part of original coco

In [92]:
# C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017

images_path = r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\images\train2017"
label_path = r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\coco2017\labels\train2017"


images_files = os.listdir(images_path)

for names in images_files: 
  if names[0:4] != "0000": # original coco images are 0000
    jpg_path = os.path.join(images_path,names)
    label_names = names.split(".")[0] + ".txt"
    txt_path = os.path.join(label_path,label_names)
    os.remove(jpg_path)
    os.remove(txt_path)


### Combining Jacks Yolo


In [29]:
from string import punctuation
any(p in "my_text" for p in punctuation)

True

In [32]:
import os 
import shutil
jack_image_path = os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\images\train")
jack_label_path = os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\labels\train")

numbers = [str(i) for i in range(0,10)]

print(jack_image_path)
train_files = os.listdir(jack_image_path)
# print(train_files)

annotations= []
for file_names in [file for file in train_files if file.endswith(".jpg")]: # for each line
  f_name=os.path.splitext(file_names)[0]
  if not any(p in f_name for p in punctuation):
    image_line = f"./images/train2017/{file_names}"
    annotations.append(image_line) # List of string for each line (replace first value with class number entered)



print(os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\train.txt"))
with open(os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\train.txt"), 'w') as outfile:
  for line in annotations:
      outfile.write(line)
      outfile.write("\n")
outfile.close()


C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\images\train
C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\train.txt


In [53]:
# Adjust head label to be 84
obj = 'head'
current_class_number = 85
new_class_number = 84
jack_label_path = os.path.normpath(r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\labels\train")
print(jack_label_path)

for file_name in os.listdir(jack_label_path): # for each line
  text_file = os.path.join(jack_label_path,file_name)
  annotations = []
  with open(text_file) as f: # open text file
      for line in f: # for each line
        labels = line.split() 
        if (not labels[0].isdigit()):
          raise Exception(text_file, "label is not a digit (possible still a label not a number)") 
        if (int(labels[0])) == current_class_number:
          labels[0] = new_class_number
          newline = str(labels[0]) + " " + str(labels[1]) + " " + str(labels[2]) + " " + str(labels[3]) + " " + str(labels[4])
          line = line.replace(line, newline)
          annotations.append(line) # List of string for each line (replace first value with class number entered)
        else: 
          annotations.append(line) # List of string for each line (replace first value with class number entered)
      f.close()

  with open(text_file, "w") as outfile:
    for line in annotations:
        outfile.write(line)
    outfile.close()

C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\jack_dataset\coco_skateboard_bicycle\labels\train


In [26]:
dataset="validation"
object_name = "Vehicle_registration_plate"
object_folder = object_name.lower().replace(" ", "_")  # new folder name

# Get the images in the object folder
oidv_obj_path = os.path.join( "OID", "Dataset", dataset, object_name)
print(oidv_obj_path)

OID\Dataset\validation\Vehicle_registration_plate


In [43]:
adjust_label_folder= r"C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\airport\motorcycle\train\labels"
print(adjust_label_folder)


file_names =  [
        os.path.join(adjust_label_folder,file) for file in os.listdir(adjust_label_folder) if file.endswith(".txt")
    ]

print(file_names[0])

def adjust_label(class_number: int, text_file: str) -> None:
    annotations = []
    with open(text_file) as f:
        for line in f:
            labels = line.split()
            labels[0] = class_number  # Change the class number
            newline = (
                str(labels[0])
                + " "
                + str(labels[1])
                + " "
                + str(labels[2])
                + " "
                + str(labels[3])
                + " "
                + str(labels[4])
            )
            line = line.replace(line, newline)
            annotations.append(line)
    print(annotations)
    with open(r"C:\Users\justm\OneDrive\Desktop\leave.txt", "w") as outfile:
        for line in annotations:
            outfile.write(line)
            outfile.write("\n")
    outfile.close()

adjust_label(1,file_names[0])


C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\airport\motorcycle\train\labels
C:\Users\justm\Documents\OID_YoloV5\yolov5\datasets\airport\motorcycle\train\labels\motorcycle_1.txt
['1 0.0630864375 0.48562341690962096 0.12421800000000001 0.284714', '1 0.7498044375 0.5265637551020408 0.49843799999999994 0.9416569999999999']
